In [18]:
from copy import deepcopy

import numpy as np
from numpy.random import randint

from utilities import create_laplace_profile, profile_most_probable_kmer, score_motifs

In [13]:
t=10
k=4
randint(0, size=10, high=t-k+1)

array([3, 5, 4, 3, 3, 0, 2, 0, 4, 4])

In [14]:
def select_random_motifs(dna, k):
    t = len(dna)
    motif_start_indices = randint(0, size=t, high=len(dna[0])-k+1)
    motifs = []
    for i, start in enumerate(motif_start_indices):
        motifs.append(dna[i][start:start+k])
    return motifs
 

In [32]:
def randomized_motif_search_inner(dna, k):
    motifs = select_random_motifs(dna, k)
    best_motifs = deepcopy(motifs)
    best_score = score_motifs(best_motifs)
    while True:
        profile = create_laplace_profile(motifs)
        motifs = []
        for dna_string in dna:
            motif = profile_most_probable_kmer(dna_string, k, profile)
            motifs.append(motif)
        score = score_motifs(motifs)
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
        else:
            return best_motifs
        
def randomized_motif_search(dna, k, n_iter=1000):
    best_motifs = randomized_motif_search_inner(dna, k)
    best_score = score_motifs(best_motifs)
    for i in range(n_iter-1):
        if i%10==0:
            print(i)
        motifs = randomized_motif_search_inner(dna, k)
        score = score_motifs(motifs)        
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
    return best_motifs

In [33]:
k = 8 
dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
]

In [34]:
expected = [
    "TCTCGGGG",
    "CCAAGGTG",
    "TACAGGCG",
    "TTCAGGTG",
    "TCCACGTG"
]

assert randomized_motif_search(dna, k) == expected

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [37]:
with open("datasets/dataset_161_5.txt","r") as fin:
    dna = []
    for i, line in enumerate(fin):
        if i==0:
            k, _ = line.strip().split(" ")
            k = int(k)
        else:
            dna.append(line.strip())

In [ ]:
for motif in randomized_motif_search(dna, k, n_iter=600):
    print(motif)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
